## Telco Customer Churn – Data Cleaning Notebook

### Description:
This notebook performs a reproducible data cleaning pipeline for the Telco Customer Churn dataset.
It includes: loading the data, inspecting it, dropping unwanted columns, converting types, imputing missing values, encoding categorical variables (binary + top-N strategy), scaling numeric features, and saving a cleaned CSV ready for modeling.

Target column: Churn Value (0 = No, 1 = Yes)
Dropped columns : Churn Label, Lat Long, Latitude, Longitude, Count, Zip Code

### Objectives

- Load the Telco dataset.
- Inspect structure, types, and missing values.
- Remove specified columns.
- Convert Total Charges to numeric.
- Impute missing values (median for numeric, mode for categorical).
- Encode categorical variables:
- Map binary-like columns to 0/1.
- One-hot encode low-cardinality categoricals (≤ 20 unique values).
- For high-cardinality categoricals, keep top-10 categories and mark others as Other then one-hot encode.
- Scale numeric features (StandardScaler).
- Export cleaned CSV: /Cleandata/cleaned_customer_churn.csv

In [48]:
import pandas as pd
import numpy as np
INPUT_PATH = "RawData/Telco_customer_churn.csv"
OUTPUT_PATH = "RawData/cleaned_customer_churn.csv"


In [49]:
df = pd.read_csv(INPUT_PATH)
print("Loaded dataset shape:", df.shape)
display(df.head())

Loaded dataset shape: (7043, 33)


,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


### Initial inspection

In [50]:
print("Columns:\n", df.columns.tolist())
print("\nData types:\n", df.dtypes.value_counts())
print("\nMissing values (count and percent):")
missing = pd.concat([df.isnull().sum(), (df.isnull().mean()*100)], axis=1)
missing.columns = ["missing_count", "missing_percent"]
display(missing.sort_values(by="missing_count", ascending=False))

Columns:
 ['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value', 'Churn Score', 'CLTV', 'Churn Reason']

Data types:
 object     24
int64       6
float64     3
Name: count, dtype: int64

Missing values (count and percent):


,missing_count,missing_percent
Churn Reason,5174,73.463013
CustomerID,0,0.000000
Count,0,0.000000
State,0,0.000000
Country,0,0.000000
Zip Code,0,0.000000
Lat Long,0,0.000000
Latitude,0,0.000000
City,0,0.000000
Gender,0,0.000000


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [52]:
print("Duplicates:", df.duplicated().sum())


Duplicates: 0


In [53]:
# remove duplicates rows
df = df.drop_duplicates()
print("Duplicates:", df.duplicated().sum())


Duplicates: 0


In [54]:
# Drop unneeded columns

cols_to_drop = ["Churn Label", "Lat Long", "Latitude", "Longitude", "Count", "Zip Code"]
cols_present_to_drop = [c for c in cols_to_drop if c in df.columns]
df.drop(columns=cols_present_to_drop, inplace=True, errors="ignore")
print(df.shape)
df.head()



(7043, 27)


,CustomerID,Country,State,City,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,...,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,United States,California,Los Angeles,Male,No,No,No,2,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,86,3239,Competitor made better offer
1,9237-HQITU,United States,California,Los Angeles,Female,No,No,Yes,2,Yes,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,67,2701,Moved
2,9305-CDSKC,United States,California,Los Angeles,Female,No,No,Yes,8,Yes,...,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,1,86,5372,Moved
3,7892-POOKP,United States,California,Los Angeles,Female,No,Yes,Yes,28,Yes,...,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1,84,5003,Moved
4,0280-XJGEX,United States,California,Los Angeles,Male,No,No,Yes,49,Yes,...,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,1,89,5340,Competitor had better devices


In [55]:
# Convert Total Charges to numeric
if "Total Charges" in df.columns:
   df["Total Charges"] = pd.to_numeric(df["Total Charges"], errors="coerce")
print("Converted 'Total Charges' to numeric. NaNs after conversion:", df["Total Charges"].isnull().sum())

Converted 'Total Charges' to numeric. NaNs after conversion: 11


In [56]:
print(df.isnull().sum())

CustomerID              0
Country                 0
State                   0
City                    0
Gender                  0
Senior Citizen          0
Partner                 0
Dependents              0
Tenure Months           0
Phone Service           0
Multiple Lines          0
Internet Service        0
Online Security         0
Online Backup           0
Device Protection       0
Tech Support            0
Streaming TV            0
Streaming Movies        0
Contract                0
Paperless Billing       0
Payment Method          0
Monthly Charges         0
Total Charges          11
Churn Value             0
Churn Score             0
CLTV                    0
Churn Reason         5174
dtype: int64


In [57]:
#Separate column types

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

numeric_features = [c for c in numeric_cols]
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numeric features:", numeric_features)
print("Categorical columns:", categorical_cols)


#Impute missing values
#Numeric -> median, Categorical -> mode

for c in numeric_features:
    if df[c].isnull().any():
        median_val = df[c].median()
        df[c].fillna(median_val, inplace=True)
        print(f"Imputed numeric '{c}' with median: {median_val}")

for c in categorical_cols:
    if df[c].isnull().any():
        mode_val = df[c].mode().iloc[0]
        df[c].fillna(mode_val, inplace=True)
        print(f"Imputed categorical '{c}' with mode: {mode_val}")

Numeric features: ['Tenure Months', 'Monthly Charges', 'Total Charges', 'Churn Value', 'Churn Score', 'CLTV']
Categorical columns: ['CustomerID', 'Country', 'State', 'City', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Churn Reason']
Imputed numeric 'Total Charges' with median: 1397.475
Imputed categorical 'Churn Reason' with mode: Attitude of support person


C:\Users\dell\AppData\Local\Temp\ipykernel_23780\3627039448.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(median_val, inplace=True)
C:\Users\dell\AppData\Local\Temp\ipykernel_23780\3627039448.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [58]:
print(df.isnull().sum())

CustomerID           0
Country              0
State                0
City                 0
Gender               0
Senior Citizen       0
Partner              0
Dependents           0
Tenure Months        0
Phone Service        0
Multiple Lines       0
Internet Service     0
Online Security      0
Online Backup        0
Device Protection    0
Tech Support         0
Streaming TV         0
Streaming Movies     0
Contract             0
Paperless Billing    0
Payment Method       0
Monthly Charges      0
Total Charges        0
Churn Value          0
Churn Score          0
CLTV                 0
Churn Reason         0
dtype: int64


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Country            7043 non-null   object 
 2   State              7043 non-null   object 
 3   City               7043 non-null   object 
 4   Gender             7043 non-null   object 
 5   Senior Citizen     7043 non-null   object 
 6   Partner            7043 non-null   object 
 7   Dependents         7043 non-null   object 
 8   Tenure Months      7043 non-null   int64  
 9   Phone Service      7043 non-null   object 
 10  Multiple Lines     7043 non-null   object 
 11  Internet Service   7043 non-null   object 
 12  Online Security    7043 non-null   object 
 13  Online Backup      7043 non-null   object 
 14  Device Protection  7043 non-null   object 
 15  Tech Support       7043 non-null   object 
 16  Streaming TV       7043 

In [60]:
# Binary encoding (common mappings)

binary_map = {
"Yes": 1, "No": 0,
"Male": 1, "Female": 0,
"no": 0, "yes": 1, # lower-case safety
"No phone service": 0,
"No internet service": 0
}

# Apply mapping for columns with exactly two unique values (after imputation)

for c in list(categorical_cols): # copy to avoid modification issues
    uniques = df[c].dropna().unique()
    if len(uniques) == 2:
    # attempt map; fallback to label encoding if mapping not found
        mapped = df[c].map(binary_map)
        if mapped.isnull().all():
        # fallback numeric label encoding
            df[c] = pd.factorize(df[c])[0]
            print(f"Label-encoded binary column '{c}' (factorize).")
        else:
            df[c] = mapped.fillna(df[c])
            print(f"Mapped binary-like column '{c}' using binary_map.")

Mapped binary-like column 'Gender' using binary_map.
Mapped binary-like column 'Senior Citizen' using binary_map.
Mapped binary-like column 'Partner' using binary_map.
Mapped binary-like column 'Dependents' using binary_map.
Mapped binary-like column 'Phone Service' using binary_map.
Mapped binary-like column 'Paperless Billing' using binary_map.


In [61]:
# save data after cleaning and
df.to_csv(OUTPUT_PATH, index=False)
print(f"✅ Cleaned dataset saved to: {OUTPUT_PATH}")

✅ Cleaned dataset saved to: RawData/cleaned_customer_churn.csv


In [ ]:
# Column Name	                                              Description
# CustomerID	                                              Unique identifier for each customer	
# Count	                                                      Likely indicates number of connections or linked services (often always 1 in many datasets)	
# Country / State / City / Zip Code	                          Geographic location of the customer	
# Lat Long / Latitude / Longitude	                          Precise geographic coordinates	
# Gender	Male / Female	
# Senior Citizen	                                          Indicates if customer is a senior (e.g., age 65+)	Seniors may churn differently than younger customers
# Partner	                                                  Whether the customer has a partner/spouse	
# Dependents	                                              Whether others rely on the customer (e.g., children)	
# Tenure	                                                  Number of months the customer has stayed with the company  
# Phone Service	                                              Whether they subscribed to basic phone service	
# Multiple Lines	                                          Indicates if they have more than one phone line	Customers with multiple services are more likely to stay
# Internet Service	Type of internet (DSL / Fiber / None)	
# Online Security / Online Backup / Device Protection / Tech Support / Streaming TV / Streaming Movies	Optional add-on services	
# Contract	Month-to-month / One year / Two years	
# Paperless Billing	Yes / No                                digital or paper invoices	
# Payment Method	                                        Electronic check, credit card, bank transfer, etc.	
# Monthly Charges	                                        Amount billed per month	High monthly cost customers might be more likely to leave
# Total Charges	                                            Lifetime revenue from the customer	
# Churn Label	Yes / No —                                  Did the customer leave?	Text-based target column
# Churn Value	0 / 1                                       numerical version of churn	Numeric target column (better for modeling)
# Churn Score	                                            Predefined churn probability score from another system	
# CLTV (Customer Lifetime Value)	                        Estimated business value of this customer in the long-term	
# Churn Reason	                                            Stated reason for leaving (if churned)	